# AlaTB example for ff03-star force field - parallel 
In this example we run OpenMM simulations of the alanine dipeptide in parralel taking advantage of functionalities of the `multiprocessing` library.

In [1]:
import timeit
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import multiprocessing as mp
import multiprocessing.pool as pool

### Read GMX files
Read topology and coordinates from GMX files

In [2]:
gro = GromacsGroFile('alaTB_ff03-star_tip3p_solv.gro')

In [3]:
top = GromacsTopFile('alaTB_ff03-star_tip3p.top', \
        periodicBoxVectors=gro.getPeriodicBoxVectors(),
        includeDir='.')

### Simulation workflow
Here we will start with a standard simulation workflow, by first energy minimizing, then running short NVT with constraints, followed by a short NPT run and a production run.

In [4]:
system = top.createSystem(nonbondedMethod=PME, \
        nonbondedCutoff=1*nanometer,
        constraints=HBonds)

#### Defining position restraints
We add position restraints in the heavy atoms using the ```CustomExternalForce``` function.

In [5]:
force = CustomExternalForce(\
            "k*((x - x0)^2 + (y - y0)^2 + (z - z0)^2)")
force.addPerParticleParameter("k")
force.addPerParticleParameter("x0")
force.addPerParticleParameter("y0")
force.addPerParticleParameter("z0")
for i, atom_crd in enumerate(gro.positions):
    k = 1000.0*kilojoules_per_mole/nanometer**2
    x0 = atom_crd[0]
    y0 = atom_crd[1]
    z0 = atom_crd[2]
    if (gro.atomNames[i][0] in ('C', 'N', 'O')) \
            and (gro.atomNames[i][0] not in ('OW')):
        force.addParticle(i,[k, x0, y0, z0])
    else:
        force.addParticle(i,[0., x0, y0, z0])
system.addForce(force)

5

Next we generate the integrator with a ```LangevinIntegrator``` object and pass it on to a ```Simulation``` object. 

In [6]:
integrator = LangevinIntegrator(300*kelvin, \
                        1/picosecond, \
                        0.002*picoseconds)

In [7]:
nvt_posre = Simulation(top.topology, system, \
                        integrator)

We use the gro file atomic positions as initial positions for the simulation.

In [8]:
nvt_posre.context.setPositions(gro.positions)

First, we run a quick energy minimization.

In [9]:
nvt_posre.minimizeEnergy()

We define the reporters for both time series of conformations and also for energies.

In [10]:
pdb_rep = PDBReporter('nvt_posre.pdb', 1000)
nvt_posre.reporters.append(pdb_rep)
state_rep = StateDataReporter(stdout, 100, \
                step=True, potentialEnergy=True, \
                temperature=True, volume=True, density=True)
nvt_posre.reporters.append(state_rep)

In [11]:
nvt_posre.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Density (g/mL)"
100,-32566.829018115124,56.227451694288554,20.679574978308164,0.9753715458022438
200,-31771.602455615124,104.14565485450024,20.679574978308164,0.9753715458022438
300,-31102.938393115124,139.17324309502564,20.679574978308164,0.9753715458022438
400,-30565.899330615124,164.07844989658443,20.679574978308164,0.9753715458022438
500,-30155.993080615124,184.75299337302414,20.679574978308164,0.9753715458022438
600,-29685.727455615124,200.5334474708224,20.679574978308164,0.9753715458022438
700,-29335.868080615124,215.95314213987598,20.679574978308164,0.9753715458022438
800,-29164.336830615124,233.92856926006868,20.679574978308164,0.9753715458022438
900,-28882.618080615124,242.18781913093596,20.679574978308164,0.9753715458022438
1000,-28690.227455615124,253.1894944440967,20.679574978308164,0.9753715458022438


#### NPT run
Next we run the NPT simulation. First we remove the position restrains from the system, because we have already equilibrated the water.

In [12]:
nforces = len(system.getForces())
system.removeForce(nforces-1)

We then grab positions and velocities from the previous simulation.

In [13]:
state = nvt_posre.context.getState(getPositions=True, \
                    getVelocities=True)

For NPT we must define a barostat that keeps control of the pressure.

In [14]:
barostat = MonteCarloBarostat(1.0*bar, 300.0*kelvin, 25)

In [15]:
system.addForce(barostat)

5

The remaining steps are very similar to those in the NVT simulation with position restraints.

In [16]:
integrator = LangevinIntegrator(300*kelvin, \
                        10/picosecond, \
                        0.002*picoseconds)

In [17]:
npt = Simulation(top.topology, system, \
                        integrator)
pdb_rep = PDBReporter('npt.pdb', 1000)
nvt_posre.reporters.append(pdb_rep)

npt.context.setState(state)
npt.reporters.append(state_rep)
npt.step(1000)

100,-27760.90020167915,283.9497725093894,20.884919935707988,0.9657814861257725
200,-27193.08472448401,308.0644596573638,21.06613247253637,0.9574737574360904
300,-27183.081501104054,289.54513733092733,20.80250476094031,0.9696077104617908
400,-27084.685454163584,297.99887959502746,20.871406697106597,0.9664067834930378
500,-26879.74494899655,304.7382879005271,20.845012201694754,0.9676304728421286
600,-26969.105253101705,303.9223584787869,20.779907329753595,0.9706621253428377
700,-26808.903246271046,294.81889358625637,20.81212286689072,0.9691596163510071
800,-26714.68901268096,305.9405269794718,20.957716275136818,0.9624268574078766
900,-26770.996415669826,309.4134659239237,20.71807538943384,0.973559012311183
1000,-26938.94482509687,306.5914608036112,20.699352062029156,0.9744396323460834


We save the coordinates of the final step in `xml` format.

In [18]:
npt.saveState("npt.xml")

### Production NVT
This is the part we are changing here. Instead of simply running a simulation we are generating inputs for two identical runs. Again, we first remove the last force added, in this case the barostat.

In [19]:
nforces = len(system.getForces())
system.removeForce(nforces-1)

Here is where things change. We are starting one of the runs from the coordinates corresponding to the `xml` file we saved (`npt.xml`), and the other one from the `State` class object we kept in `npt`. For the former we only need the coordinates of the `xml` file, which we read from `mdtraj`.

In [21]:
import mdtraj as md
traj = md.load_xml("npt.xml", top='alaTB_ff03-star_tip3p_solv.gro')
last_frame = traj[-1].xyz[0]

Now comes the `multiprocessing` stuff, which is fairly simple. We just need to open a `ThreadPool` for whichever number of processes we want (here it is fixed as the number of processors). Then we prepare the runs using the `prepare_runs` next

In [24]:
def prepare_runs(init):
    """
    Prepares runs for OpenMM
    
    Parameters
    ----------
    init : list
        List containing indexes and npt input for runs.
        init[0] : index
        init[1] : initial state or configuration
 
    Returns
    -------
    nvt : 
        Simulation object with all information to simply be run.
    
    """
    ind = init[0]
    npt = init[1]
    
    integrator = LangevinIntegrator(300*kelvin, \
                        10/picosecond, \
                        0.002*picoseconds)
    
    nvt = Simulation(top.topology, system, \
                        integrator)
    
    try:
        state = npt.context.getState(getPositions=True, \
                    getVelocities=True)   
        nvt.context.setState(state)
    except AttributeError as e:
        nvt.context.setPositions(npt)
        nvt.context.setVelocitiesToTemperature(300*kelvin)

    dcd_rep = DCDReporter('nvt%i.dcd'%ind, 1000)
    nvt.reporters.append(dcd_rep)
    state_rep = StateDataReporter("nvt%i.txt"%init[0], 100, \
                step=True, potentialEnergy=True, \
                temperature=True, volume=True, density=True)
    nvt.reporters.append(state_rep)

    return nvt

In [25]:
nproc = mp.cpu_count()

In [26]:
tpool = pool.ThreadPool(processes=nproc)
mpinput = [(i,x) for i,x in enumerate([npt, last_frame])]
result = tpool.map(prepare_runs, mpinput)
tpool.close()
tpool.join()

`result` contains the output, which here are two `Simulation` objects that we need to run.

In [71]:
def do_run(x):
    """
    Run the simulation
    
    Parameters:
    ----------
    x : simtk.openmm.app.simulation.Simulation
        Simulation object from OpenMM

    Returns:
    --------
    elapsed : float
        Elapsed time between beginning and end.
    
    """
    start = timeit.timeit()
    x.step(10000)
    end = timeit.timeit()
    elapsed = end-start
    return elapsed

In [72]:
tpool = pool.ThreadPool(processes=nproc)
elapsed = tpool.map(do_run, result)
tpool.close()
tpool.join()

0.01712620099715423 0.016089388998807408
0.011813154997071251 0.015181976996245794


In [27]:
result